## Pre-Process the data

In [27]:
import pandas as pd

reviews = pd.read_csv('data/BA_reviews.csv')

In [28]:
reviews.head()

,Unnamed: 0,reviews
0,0,✅ Trip Verified | Manchester to Copenhagen vi...
1,1,✅ Trip Verified | I have never seen such disr...
2,2,✅ Trip Verified | Paid for a 14 hour long fli...
3,3,✅ Trip Verified | Very inconsiderate PA annou...
4,4,"✅ Trip Verified | Absolutely terrible, lost m..."


In [29]:
reviews.columns = ['Rating', 'Review']

In [30]:
reviews.head()

,Rating,Review
0,0,✅ Trip Verified | Manchester to Copenhagen vi...
1,1,✅ Trip Verified | I have never seen such disr...
2,2,✅ Trip Verified | Paid for a 14 hour long fli...
3,3,✅ Trip Verified | Very inconsiderate PA annou...
4,4,"✅ Trip Verified | Absolutely terrible, lost m..."


In [31]:
reviews['Review'] = reviews['Review'].apply(lambda x: x.replace("✅ Trip Verified |","")) 

In [32]:
reviews.head()

,Rating,Review
0,0,Manchester to Copenhagen via London on 30th ...
1,1,I have never seen such disrespect to a custo...
2,2,Paid for a 14 hour long flight ticket which ...
3,3,Very inconsiderate PA announcements were mad...
4,4,"Absolutely terrible, lost my luggage on both..."


In [33]:
reviews['Review'] = reviews['Review'].apply(lambda x: x.replace("Not Verified |","")) 

In [34]:
reviews.head()

,Rating,Review
0,0,Manchester to Copenhagen via London on 30th ...
1,1,I have never seen such disrespect to a custo...
2,2,Paid for a 14 hour long flight ticket which ...
3,3,Very inconsiderate PA announcements were mad...
4,4,"Absolutely terrible, lost my luggage on both..."


In [37]:
reviews.Review.unique().shape

(1000,)

1000 Unique Reviews

## Topic Modelling
Proceed on the basis that data is cleaned and pre-processed

Reference:
https://ourcodingclub.github.io/tutorials/topic-modelling-python/

In [38]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+') # transformer to vector form
# max df discards words that are in 90% of reviews - too common
# min df ensures that words in less than 25 reviews will be discarded - no correlation too minor
tf = vectorizer.fit_transform(reviews.toarray()) # applies transformer to our data
tf_feature_names = vectorizer.get_feature_names() # provides insight as to what word each column in matrix represent

AttributeError: 'DataFrame' object has no attribute 'toarray'

In [ ]:
tf_feature_names
# word frequency : review

Let's now create the model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=18)
model.fit(tf)

## Sentiment Analysis
Reference:
https://realpython.com/python-nltk-sentiment-analysis/

In [ ]:
# Write Code Here

## Word Cloud
Reference:
https://www.projectpro.io/recipes/create-word-cloud-python

In [ ]:
# Write Code Here